![Factories creating emissions](assets/images/pollution.jpg)

Photo by Maxim Tolchinskiy on Unsplash
  
When factoring heat generation required for the manufacturing and transportation of products, _Greenhouse gas emissions attributable to products, from food to sneakers to appliances, make up more than 75% of global emissions._ (`Source: The Carbon Catalogue https://www.nature.com/articles/s41597-022-01178-9`)

Our data, which is publicly available on nature.com, contains product carbon footprints (PCFs) for various companies. PCFs are the greenhouse gas emissions attributable to a given product, measured in CO<sub>2</sub> (carbon dioxide equivalent).
<!--https://www.nature.com/articles/s41597-022-01178-9-->

This data is stored in a PostgreSQL database containing one table, `product_emissions`, which looks at PCFs by product as well as the stage of production that these emissions occurred. Here's a snapshot of what `product_emissions` contains in each column:

### `product_emissions`

| field                              | data type |
|------------------------------------|-----------|
| `id`                                 | `VARCHAR`   |
| `year`                               | `INT`       |
| `product_name`                       | `VARCHAR`   |
| `company`                            | `VARCHAR`   |
| `country`                            | `VARCHAR`   |
| `industry_group`                     | `VARCHAR`   |
| `weight_kg`                          | `NUMERIC`   |
| `carbon_footprint_pcf`               | `NUMERIC`   |
| `upstream_percent_total_pcf`         | `VARCHAR`   |
| `operations_percent_total_pcf`       | `VARCHAR`   |
| `downstream_percent_total_pcf`       | `VARCHAR`   |

You'll use this data to examine the carbon footprint of each industry in the dataset! 

In [10]:
import duckdb
import pandas as pd
df = pd.read_csv("ds/product_emissions.csv")

In [11]:
#Update your query here
consulta ="""
SELECT *
FROM df
LIMIT 5;
"""
rsl=duckdb.sql(consulta).to_df()
rsl

,index,id,year,product_name,company,country,industry_group,weight_kg,carbon_footprint_pcf,upstream_percent_total_pcf,operations_percent_total_pcf,downstream_percent_total_pcf
0,0,10056-1-2014,2014,Frosted Flakes(R) Cereal,Kellogg Company,USA,"Food, Beverage & Tobacco",0.7485,2.00,57.50%,30.00%,12.50%
1,1,10056-1-2015,2015,"Frosted Flakes, 23 oz, produced in Lancaster, ...",Kellogg Company,USA,Food & Beverage Processing,0.7485,2.00,57.50%,30.00%,12.50%
2,2,10222-1-2013,2013,Office Chair,KNOLL INC,USA,Capital Goods,20.6800,72.54,80.63%,17.36%,2.01%
3,3,10261-1-2017,2017,Multifunction Printers,"Konica Minolta, Inc.",Japan,Technology Hardware & Equipment,110.0000,1488.00,30.65%,5.51%,63.84%
4,4,10261-2-2017,2017,Multifunction Printers,"Konica Minolta, Inc.",Japan,Technology Hardware & Equipment,110.0000,1818.00,25.08%,4.51%,70.41%


In [12]:
consulta = """
WITH recent_year AS (
    SELECT MAX(year) AS last_year
    FROM df
)
SELECT 
    industry_group, 
    COUNT(DISTINCT company) AS num_companies, 
    ROUND(SUM(carbon_footprint_pcf), 1) AS total_industry_footprint 
FROM df 
WHERE year = (SELECT last_year FROM recent_year) 
GROUP BY industry_group 
ORDER BY total_industry_footprint DESC;
"""

resultado = duckdb.sql(consulta).to_df()  # Convertir a DataFrame de pandas
resultado

,industry_group,num_companies,total_industry_footprint
0,Materials,3,107129.0
1,Capital Goods,2,94942.7
2,Technology Hardware & Equipment,4,21865.1
3,"Food, Beverage & Tobacco",1,3161.5
4,Commercial & Professional Services,1,740.6
5,Software & Services,1,690.0
